# 방금 만든 모델로 인퍼런스를 진행하기 위해 필요한 것을 다운받습니다.

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 4.9 MB 4.2 MB/s 
     |████████████████████████████████| 6.6 MB 34.6 MB/s 
     |████████████████████████████████| 120 kB 44.7 MB/s 


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, AutoTokenizer
import numpy as np
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
#id2label = {0: '활동', 1: '숙소', 2: '맛집', 3: '날씨', 4: '카페', 5: '인사', 6: '관광지추천', 7: '소개'}
#id2label = {0: '날씨 묻기', 1: '관광지 추천', 2: '숙소 추천', 3: '맛집 추천', 4: '인사', 5: '소개', 6:'활동 추천', 7:'카페 추천'}
id2label = {0: '관광지 추천', 1: '날씨 묻기', 2: '맛집 추천', 3: '숙소 추천', 4: '인사', 5: '카페 추천', 6: '활동 추천'}

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 이전 파일에서 저장했던 모델을 불러옵니다.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/중정처1주차/my_model', local_files_only=True)
trainer = Trainer(model=model)

# Inference

모델에 새로운 문장을 넣어 잘 작동하는지 확인합니다.

In [ ]:
text = "넌 기능이 뭐야?"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [ ]:
encoding

{'input_ids': tensor([[   2,  746, 4039, 2052, 1097, 2275,   35,    3]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

모델의 결과값으로 나오는 logits의 shape은 (batch_size, num_labels) 입니다. 우리는 하나의 문장만 모델에 입력하기 때문에 batch_size가 1이 되어야 합니다.

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 7])

predict된 점수들 중 가장 높은 점수를 1로 변경해준 다음 앞에서 만들었던 사전을 이용해 텍스트로 변경해줍니다.

In [ ]:
probs = logits.squeeze().cpu()
predictions = np.where(probs==probs.max(), 1, 0)
print(predictions)
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

[0 0 0 0 0 0 1]
['활동 추천']


# 플라스크로 모델을 웹사이트에 띄워보기 (데모)

## Step1. 웹사이트에 사용될 inference 함수를 만듭니다

In [ ]:
# Q1. 하나의 문장을 입력하면[input] 주제(라벨)을 반환[output]하는 함수 'inference' 를 만들어보세요:

def inference()

SyntaxError: ignored

### 답

In [ ]:
def inference(text):
  encoding = tokenizer(text, return_tensors = 'pt')
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

  outputs = trainer.model(**encoding)

  logits = outputs.logits

  probs = logits.squeeze().cpu()
  predictions = np.where(probs==probs.max(), 1, 0)
  predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]

  return predicted_labels[0]

#### 4팀의 함수입니다
#### softmax를 쓴 이유?: 기타를 분류하기 위해서 

* <font color=blue>probs.max와 softmax의 차이점</font>


기존 함수의 경우 무조건 probs.max()로 최대 확률 라벨을 가져오기 때문에,
어떤 특정 라벨로 분류할 수 없는 경우에도 예외 없이 최대 확률을 가져온다.
그런 경우들을 "라벨을 분류해야 하는 경우"와 구분하지 못한다.

softmax threshold를 쓰면, 라벨을 분류할 수 없는 경우를 threshold 밑으로 가도록 설정하여, (여기서는 0.97 미만으로), 해당 경우들을 하나의 조건으로 분류할 수 있다.

In [ ]:
# softmax 와 threshold 를 사용하는 새로운 인퍼런스 함수
import torch
import torch.nn.functional as F

def inference(text):
  encoding = tokenizer(text, return_tensors = 'pt')
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

  outputs = trainer.model(**encoding)

  logits = outputs.logits

  probs = logits.squeeze().cpu()
  softmax_probs = F.softmax(probs, dim=-1)
  labels_predict = np.where(softmax_probs==softmax_probs.max())[0][0]

  if softmax_probs[labels_predict] < 0.97:
    return '기타'
  else:
    return id2label[labels_predict]

### 내가 만든 함수 사용

In [ ]:
# Q2. 내가 만든 함수를 사용해보세요:
inference("호텔 어디가 좋냐")

'기타'

## Step2. 플라스크 라이브러리 및 웹 실행을 위해 필요한 것들을 설치해줍니다.

In [ ]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
print("Completed!")

Completed!


In [ ]:
# install ngrok linux version using the following command or you can get the
# latest version from its official website- https://dashboard.ngrok.com/get-started/setup

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

--2022-09-24 01:58:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  19.3MB/s    in 0.7s    

2022-09-24 01:58:43 (19.3 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]



In [ ]:
# extract the downloaded file using the following command 

!tar -xvf /content/ngrok-stable-linux-amd64.tgz

ngrok


In [ ]:
# ngrok 사이트 (https://dashboard.ngrok.com/get-started/setup)에서 회원가입을 한뒤
# "Your Authtoken" 탭을 클릭하여 authtoken을 복사하여 붙여넣습니다.

!./ngrok authtoken 2E1OM4jFIMoCDY1wJW7ssSHkft8_hDR6HZQhxvb7ijtysao

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# import Flask from flask module
from flask import Flask
from flask import render_template, request

# import run_with_ngrok from flask_ngrok to run the app using ngrok
from flask_ngrok import run_with_ngrok

다음 셀을 실행시킨 후,

세 개의 주소 중 ngrok.io 으로 끝나는 주소를 클릭하여 결과를 확인해보세요!

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def man():
  return """
  <html>
    <body bgcolor=#F5F5DC>                                   <!-- bgcolor: 배경색: 베이지 색 -->             

        <center>                                             <!-- 가운데 정렬 -->

        <h1> 내가 만든 모델과 대화해보기 </h1><br>                  <!-- <h1></h1>: 제목    <br>: 줄바꿈 -->
        
        <form method="POST", action="/predict">              <!-- <form></form>: 폼 생성 -->
                                                             <!-- <form method="POST": 폼의 method 속성.  "GET" / "POST" 선택 -->  
                                                             <!-- <form action="/predict" :   "GET" / "POST"로 받은 데이터가 도착할 URL 명시 -->  
           
           
           <b> 이 문장의 주제는? :  <input type="text", name='a', placeholder="enter 1"> <br><br>
                                                             
                                                             
                                                             <!-- <input name ="a" :  이 input을 식별할 수 있는 이름.
                                                                  밑에 보면 data1 = request.form['a']에서 이 이름을 바탕으로 정보를 요청한다. -->

            
            <input type="submit" , value='predict!' >        <!-- "predict!"의 값을 가진 정보 제출 배너 -->
        </form>
        
    </center>

    </body>
</html>
"""
    # return render_template('home.html')


@app.route('/predict', methods=['POST'])
def home():
    data1 = request.form['a']        # input name="a"로부터 정보 요청. data1 변수에 저장
    pred = inference(data1)            # inference 함수 실행
   
    return f"""
    <html>

<body bgcolor=#a3cfb4>

    <center>

        <h1> 입력하신 문장의 주제는 {pred} 입니다. </h1>       <!-- {pred}: 변수명 -->

    </center>

</body>

</html>"""

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7aac-34-125-246-31.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [24/Sep/2022 02:15:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Sep/2022 02:15:49] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Sep/2022 02:15:51] "POST /predict HTTP/1.1" 200 -


대화 시스템 예시 - bert_대화시스템 이후

In [ ]:
import torch
import torch.nn.functional as F

def answer_system(input_text):
  print("안녕하세요, 무엇을 도와드릴까요?")
  tof = answer(input_text)
  return tof

def answer(text):
  encoding = tokenizer(text, return_tensors="pt")
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
  outputs = trainer.model(**encoding)
  logits = outputs.logits
  probs = logits.squeeze().cpu()
  labels_predict = [ i if i > 0 else 0 for i in probs ]

  softmax_probs = F.softmax(probs, dim=-1)
  #print(softmax_probs)

  labels_predict = np.where(softmax_probs==softmax_probs.max())[0][0]
  pred = id2label[labels_predict]

  #print(softmax_probs[labels_predict])
  
  if softmax_probs[labels_predict] < 0.6:
    # print('unpredictable')
    pred = ''
  else:
    # print(f'predictable -> {predicted_intention}')
    pass

  
  if pred == '활동':
    print('활동')
    return True
  elif pred == '숙소':
    print('숙소')
    return True
  elif pred == '맛집':
    print('맛집')
    return True
  elif pred == '날씨':
    print('날씨')
    return True
  elif pred == '카페':
    print('카페')
    return True
  elif pred == '인사':
    print('인사')
    return True
  elif pred == '관광지추천':
    return answer_sentence('관광지추천')
    #return True
  elif pred == '소개':
    print('소개')
    return True
  else:
    print("무슨 말씀인지 모르겠습니다")
    return False

import random
def answer_sentence(intention):

  weather = ['봄', '여름', '가을', '겨울']
  choice_weather = random.choice(weather)

  attractions = ['아르떼 뮤지엄', '사려니숲길', '어승생악', '수산저수지', '원 앤 온리', '카멜리아 힐', '수월봉']
  choice_attractions = random.choice(attractions)

  sentence = f'이번 {choice_weather}, {choice_attractions} 가 보시는 건 어때요?'

  return sentence

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def man():
  return """

  <html>
    <head>
      <style>              
        #container {
          display: flex;
          justify-content: center;
          align-items: center;
        }
      </style>
    <head>

    <body bgcolor=#F5F5DC id="container">

      <center>

        <h1> 제주 관광 챗봇 </h1><br>
        <h3> 안녕하세요, 무엇을 도와드릴까요? </h3><br>
          <form method="POST", action="/predict">
            <b> 질문을 입력해주세요 :  <input type="text", name='a', placeholder="sentence"> <br><br>
              <input type="submit" , value='입력 완료' >
          </form>
      
      </center>

    </body>
</html>
"""
    # return render_template('home.html')


@app.route('/predict', methods=['POST'])
def home():
    data1 = request.form['a']
    pred = answer_system(data1)

    return f"""
      <html>
    <head>
      <style>                                    
        #container {{
          display: flex;
          justify-content: center;
          align-items: center;
          text-align: left;
          left: 50%;
        }}
      </style>
    <head>

    <body bgcolor=#F5F5DC id="container">

      <center>

        <h1> 제주 관광 챗봇 </h1><br>
        <h4> 추가 질문을 원하시면 새로 문장을 입력해주세요. </h4><br>
          <form method="POST", action="/predict">
            <b> 질문을 입력해주세요 :  <input type="text", name='a', placeholder="추가 질문"> <br><br>
              <input type="submit" , value='입력 완료' >
          </form>
      
      <h2> {pred} </h2>
      </center>

    </body>
</html>
"""

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cd5a-34-125-246-31.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [24/Sep/2022 03:08:50] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Sep/2022 03:08:50] "GET /favicon.ico HTTP/1.1" 404 -
ERROR:__main__:Exception on /predict [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule